https://pytorch.org/tutorials/advanced/super_resolution_with_caffe2.html

https://blog.exxactcorp.com/pytorch-release-v1-2-0-new-torchscript-api-with-improved-python-language-coverage-expanded-onnx-export-nn-transformer/


### Imports

In [0]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install onnx

help(torch.onnx.export)


In [0]:
#!pip install --upgrade --force-reinstall torch


In [0]:
#!pip uninstall torchvision
#!pip install -c pytorch pytorch-nightly torchvision cudatoolkit=10.0
!pip install torchvision

In [0]:
# Some standard imports
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx
import torchvision

# Super Resolution model definition in PyTorch
import torch.nn as nn
import torch.nn.init as init

In [0]:
print(torch.__version__)
print(torchvision.__version__)

1.2.0
0.4.0


### Define the paths

In [0]:
base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/' 
checkpoint_name = 'akash-mobilenet_v2-FER1-60perc.pt'
onnx_export_path = base_path + "ONNX/akash_mobilenet_60_bc2.onnx"

To export a model, you call the torch.onnx._export() function. 
This will execute the model, recording a trace of what operators are used to compute the outputs. 
Because _export runs the model, we need provide an input tensor x. 
The values in this tensor are not important; it can be an image or a 
random tensor as long as it is the right size.

In [0]:

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.


# A model class instance (class not shown)
torch_model = torchvision.models.mobilenet_v2(pretrained=False)
torch_model.classifier[1] = torch.nn.Linear(1280, 7)

#print(model)

# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None

# Load the weights from a file (.pth usually)
state_dict = torch.load(base_path + checkpoint_name, map_location=torch.device('cpu'))

# Load the weights now into a model net architecture defined by our class
torch_model.load_state_dict(state_dict)

# set the train mode to false since we will only run the forward pass.
torch_model.train(False)



In [0]:
# Create the right input shape (e.g. for an image)
dummy_input = torch.randn(1, 3, 256, 256)


torch.onnx.export(torch_model, dummy_input, onnx_export_path)

# Export the model
torch_out = torch.onnx._export(torch_model,             # model being run
                               dummy_input,             # model input (or a tuple for multiple inputs)
                               onnx_export_path,             # where to save the model (can be a file or file-like object)
                               export_params=True)      # store the trained parameter weights inside the model file

torch_out is the output after executing the model. Normally you can ignore this output, but here we will use it to verify that the model we exported computes the same values when run in Caffe2.

Now let’s take the ONNX representation and use it in Caffe2. This part can normally be done in a separate process or on another machine, but we will continue in the same process so that we can verify that Caffe2 and PyTorch are computing the same value for the network:

In [0]:
import onnx
import caffe2.python.onnx.backend as onnx_caffe2_backend

In [0]:
# Load the ONNX ModelProto object. model is a standard Python protobuf object
model = onnx.load(onnx_export_path)

# prepare the caffe2 backend for executing the model this converts the ONNX model into a
# Caffe2 NetDef that can execute it. Other ONNX backends, like one for CNTK will be
# availiable soon.
prepared_backend = onnx_caffe2_backend.prepare(model)

# run the model in Caffe2

# Construct a map from input names to Tensor data.
# The graph of the model itself contains inputs for all weight parameters, after the input image.
# Since the weights are already embedded, we just need to pass the input image.
# Set the first input.
W = {model.graph.input[0].name: dummy_input.data.numpy()}

print(model.graph.input[0])
print(model.graph.output[0])

# Run the Caffe2 net:
c2_out = prepared_backend.run(W)[0]

# Verify the numerical correctness upto 3 decimal places
np.testing.assert_almost_equal(torch_out.data.cpu().numpy(), c2_out, decimal=3)

print("Exported model has been executed on Caffe2 backend, and the result looks good!")

In [0]:
prepared_backend # caffe2.python.onnx.backend_rep.Caffe2Rep

Running the model on mobile devices
So far we have exported a model from PyTorch and shown how to load it and run it in Caffe2. Now that the model is loaded in Caffe2, we can convert it into a format suitable for running on mobile devices.

We will use Caffe2’s mobile_exporter to generate the two model protobufs that can run on mobile. The first is used to initialize the network with the correct weights, and the second actual runs executes the model. We will continue to use the small super-resolution model for the rest of this tutorial.

In [0]:
# extract the workspace and the model proto from the internal representation
c2_workspace = prepared_backend.workspace
c2_model = prepared_backend.predict_net

# Now import the caffe2 mobile exporter
from caffe2.python.predictor import mobile_exporter

# call the Export to get the predict_net, init_net. These nets are needed for running things on mobile
init_net, predict_net = mobile_exporter.Export(c2_workspace, c2_model, c2_model.external_input)

# Let's also save the init_net and predict_net to a file that we will later use for running them on mobile
with open(base_path +'ONNX/akash_mobilenet_init_net.pb', "wb") as fopen:
    fopen.write(init_net.SerializeToString())
with open(base_path + 'ONNX/akash_mobilenet_predict_net.pb', "wb") as fopen:
    fopen.write(predict_net.SerializeToString())

init_net has the model parameters and the model input embedded in it and predict_net will be used to guide the init_net execution at run-time. In this tutorial, we will use the init_net and predict_net generated above and run them in both normal Caffe2 backend and mobile and verify that the output high-resolution cat image produced in both runs is the same.

In [0]:
# Some standard imports
from caffe2.proto import caffe2_pb2
from caffe2.python import core, net_drawer, net_printer, visualize, workspace, utils

import numpy as np
import os
import subprocess
from PIL import Image
from matplotlib import pyplot
from skimage import io, transform

load the image, pre-process it using standard skimage python library. Note that this preprocessing is the standard practice of processing data for training/testing neural networks.

In [0]:
# path to test image
path_to_Test_input_img = base_path + "testface-happy.jpg"
path_to_Test_output_img = base_path + "testface_224x224.jpg"


# load the image
img_in = io.imread(path_to_Test_input_img)

# resize the image to dimensions 224x224
img = transform.resize(img_in, [256, 256])


# save this resized image to be used as input to the model
io.imsave(path_to_Test_output_img, img)

Now, as a next step, let’s take the resized  image and run the  model in Caffe2 backend and save the output image. 

In [0]:
# load the resized image and convert it to Ybr format
#img = Image.open(path_to_Test_output_img)
#img_ycbcr = img.convert('YCbCr')
#img_y, img_cb, img_cr = img_ycbcr.split()

#print(img.size)

from matplotlib.image import imread

img_np = imread(path_to_Test_output_img)
#print(type(img_np))
#print(img_np.shape)
img_np = img_np.T


# Let's run the mobile nets that we generated above so that caffe2 workspace is properly initialized
workspace.RunNetOnce(init_net)
workspace.RunNetOnce(predict_net)

# Caffe2 has a nice net_printer to be able to inspect what the net looks like and identify
# what our input and output blob names are.

#print(net_printer.to_string(predict_net))

True

From the above output, we can see that input is named “input.1” and output is named “465”(it is a little bit weird that we will have numbers as blob names but this is because the tracing JIT produces numbered entries for the models)

In [0]:
# Now, let's also pass in the resized cat image for processing by the model.
#workspace.FeedBlob("input.1", np.array(img_y)[np.newaxis, np.newaxis, :, :].astype(np.float32))

pass_img = np.broadcast_to(img_np, (1, 3, 256, 256)) 

print (pass_img.shape)

workspace.FeedBlob("input.1", np.array(pass_img).astype(np.float32))

# run the predict_net to get the model output
workspace.RunNetOnce(predict_net)

# Now let's get the model output blob
prediction = workspace.FetchBlob("465")

print(prediction)
# (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)
print(np.max(prediction))

(1, 3, 256, 256)
[[ 3.5308531e-01 -3.3968492e+00  2.0000190e-03 -6.9760841e-01
   6.6572469e-01  2.2781390e-01  4.6561572e-01]]
0.6657247


In [0]:
print(init_net.DESCRIPTOR)